In [1]:
# boilerplate imports
import time
import datetime
import math
import json
import pvlib

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from itertools import cycle

%matplotlib inline

In [2]:
########################
### GLOBAL CONSTANTS ###
########################

'''
The list of rooms, and the list of sense and control points 
should be read from a metadata store but for now they are hardcoded
'''


########## Building 1 Rooms ##########
# Interior Offices:
# interior_offices_private = [1185, 1166, 1164, 2172]
# interior_offices_shared  = [1144, 1188]

# Exterior Offices:
# exterior_offices_private = [1120, 1121, 1122, 1123, 1124, 1125, 1130, 1131, 1141, 1143, \
#                             1154, 1155, 2103, 2102, 2122, 2124, 2125, 2126, 2128, 2134, \
#                             2136, 2138, 2140, 2144, 2146, 2156, 2157, 3106, 3108, 3110, \
#                             3116, 3118, 3122, 3124, 1118, 1126, 1156, 2150, 2152, 2154, \
#                             3130, 3132, 3140, 3142, 3144, 3146, 3148]
# exterior_offices_shared  = [1156, 1118]

# Corridors: 
# corridors = [1112, 1114, 1174, 1152, 1158, 2175, 3114, 3104, \
#              2108, 2175]

# Facility Rooms:
# facility = [1136, 1163, 2180, 2117]

# Elevators:
# elevators = [2118]

# Washtooms:
# washrooms = [1169, 1167, 2169, 2171]

# SharedPlaces (Lobby, Meeting Rooms):
# open_area = [11103, 1101, 1150, 2104, 2158, 2148, 2190, 2184, 2130]
# meeting = [1108, 1106, 1104, 1162, 1165, 1142, 1128, 2016, 2170, 2142]
# conference = [2110]

# room_list = conference + meeting + open_area + washrooms + elevators + facility + \
#             corridors + exterior_offices_shared + exterior_offices_private + \
#             exterior_offices_private + interior_offices_shared + interior_offices_private

room_list = [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, \
             115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 201, 202, \
             203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, \
             217, 218, 219, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 301, \
             302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, \
             316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 327, 328, 329]


########## Building 2 Rooms ##########
# Interior Offices:
# interior_offices_private = [306, 304, 302]
# Exterior Offices:
# exterior_offices_private = [231, 229, 227, 223, 219, 217, 213, 211, 202, 203, 201, 443, 431, 429, 427, \
#                             428, 424, 421, 419, 417, 413, 411, 409, 407, 403, 341, 412, 408, 406, 404, \
#                             402, 333, 329, 327, 323, 321, 319, 339, 322, 318, 316, 314, 308, 337, 433, \
#                             317, 313, 311, 309, 307, 303]
# Corridors:
# corridors = [216, 208, 105, 226, 209, 212, \
#              101, 103, 418, 312]
# Facility Rooms:
# facility_rooms = [206, 301]
# Elevators:
# elevators = []
# Washtooms:
# washrooms = []
# SharedPlaces (Lobby, Meeting Rooms):
# shared_places = [204, 401, 205, 207]

# room_list = facility_rooms
# room_list = [106, 112, 210, 230, 312, 327, 328, 329, 303]
# room_list = [118, 303, 105, 121, 104, 106, 208]

# sensor_list = ['TL1', 'TL2', 'TL3', 'TL4', 'TL5', 'TL6', 'TL8', 'TL13']
# sensor_list = ['TL1', 'TL3', 'TL5', 'TL6', 'TL8']

sensor_list = ['TL1', 'TL3', 'TL5', 'TL8', 'TL6']

# sensor_list = ['TL1']
# sensor_list = ['TL1', 'TL13', 'TL2', 'TL3', 'TL4', 'TL5', 'TL6', 'TL8', 'TL91', 'TL92']

start_time = '2017-01-23 00:00:00'
end_time = '2017-03-06 00:00:00'

# start_time = '2017-04-10 00:00:00'
# end_time = '2017-05-01 00:00:00'

# Specify the first day

firstday = 0 # 0 -> Monday
days_number = 42


daylbl = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# daylbl = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', \
#          'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', \
#          'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', \
#          'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', \
#          'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

day_labels = daylbl[firstday:] + daylbl[:firstday]
maxnumreadings = 0

In [ ]:
def getAndFormatData(zone, stream_name, prefix):
    '''
    Gets the data for a uuid and returns a windowed resampled pandas series
    '''
    
    global maxnumreadings
    
    try:
        json_data = json.load(open('../PCL/Bldg1/' + prefix + '{}.{}.json'.format(zone, stream_name)))
        
        samples = json_data["samples"]
        # point_count = json_data["count"]
        
    except (FileNotFoundError, KeyError) as er:
        print("Error: No samples found for stream_name : %s in zone : %s" % (stream_name, zone))
        
        raise er
        # return {}
    
    else:
        values = [dp["v"] for dp in samples]
        timestamps = [dp["ts"] for dp in samples]
        ts_dt = list(map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"), timestamps))
        
        #print(ts_dt)
        
        sensor_dict = {}

        for i in range(len(ts_dt)):
            sensor_dict[ts_dt[i]] = values[i]

#         pd_sensor = pd.Series(sensor_dict).loc['2016-12-18 00:00:00':'2017-12-18 00:00:00']
        pd_sensor = pd.Series(sensor_dict).loc[start_time:end_time]
        
        if maxnumreadings < len(pd_sensor):
            maxnumreadings = len(pd_sensor)
        
        # resampled = pd_sensor.resample('10T').mean().ffill()
        resampled = pd_sensor.resample('10T').mean().ffill().bfill()
        # print(resampled)
        
    return resampled



def addToData(name, pandas_series, data, first=False):
    '''
    Formats the pandas objects into a data array
    '''
    
    indices = pandas_series.index
    for i in range(len(pandas_series)):
        if first==True:
            data[time.mktime(indices[i].to_pydatetime().timetuple())] = {}
        
        data[time.mktime(indices[i].to_pydatetime().timetuple())][name] = pandas_series[i]

    return data


def getSortedNamedStream(zone, stream_name, prefix): 
    '''
    Gets sensor data for a specific zone
    '''
    
    print("Getting data for zone : %s , stream_name : %s" % (zone, stream_name))
    
    try:
        pd_stream = getAndFormatData(zone, stream_name, prefix)
    except (FileNotFoundError, KeyError) as er:
        return ([], [])
    else:
        data = {}
        addToData(stream_name, pd_stream, data, True)
        retV = []

        for t in sorted(data):
            retV.append((t, data[t][stream_name]))

        retT = range(len(retV))
        print("Length of series %s:%s is %d " % (zone, stream_name, len(retT) ))
        retV = sorted(retV, key=lambda k:k[0])
        times = [ int(t) for t in range(len(retV))]
        values = [ v for (t,v) in retV ]
        
        return (times, values)

In [ ]:
# pulling in data

tsdata = {}

for room in room_list:
    tsdata[room] = {}
    
    for sensor in sensor_list:
        # prefix_list = ['101', '102', '103', '105', '107', '110', '120', '130', '140']
        prefix_list = ['110']
        # print(sensor)
        for prefix in prefix_list:
            temp = []
            temp.append(getSortedNamedStream(room, sensor, prefix))
            tsdata[room][sensor] = temp

Getting data for zone : 101 , stream_name : TL1
Length of series 101:TL1 is 6048 
Getting data for zone : 101 , stream_name : TL3
Length of series 101:TL3 is 6048 
Getting data for zone : 101 , stream_name : TL5


In [ ]:
for room in room_list:
    for sensor in sensor_list:
        try:
            if len(tsdata[room][sensor][0][0]) <= 0.5 * maxnumreadings + 1:
                print("Deleting sensor %s in room : %s due to missing data. Number of datapoints : %s" % (sensor, room, len(tsdata[room][sensor][0])))
                del tsdata[room][sensor]
        except:
            pass
        
black_list = []
for room in room_list:
    try:
        if (len(tsdata[room]['TL1'][0][1]) == 0):
            del tsdata[room]
            black_list.append(room)
    except:
        black_list.append(room)

In [ ]:
# temporary fix of the huge number problem:
import numpy as np

for room in room_list:
    for sensor in sensor_list:
        if (not(room in black_list)):
            try:
                if len(tsdata[room][sensor][0][1]) > 1:
                    max_inx = np.argmax(tsdata[room][sensor][0][1])
                    min_inx = np.argmin(tsdata[room][sensor][0][1])
                    tsdata[room][sensor][0][1][max_inx] = tsdata[room][sensor][0][1][min_inx]
                else:
                    max_inx = argmax(tsdata[room][sensor][0][1])
                    tsdata[room][sensor][0][max_inx] = 0
            except:
                pass

In [ ]:
# plotting

matplotlib.rcParams['figure.figsize'] = (30.0, 10.0)
colors = {'TL1':'r', 'TL3':'--m', 'TL5':'b', 'TL8':'--g', 'TL6':':c'}

def plotAllSensorData(zone, data, isSensorValue):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    
    # Move the last y-axis spine over to the right by x% of the width of the axes
    ax3.spines['right'].set_position(('axes', 1.05))
    ax3.set_frame_on(True)
    ax3.patch.set_visible(False)
    ax3.yaxis.set_major_formatter(matplotlib.ticker.OldScalarFormatter())
    
    lines = []
    line_styles = cycle(['-', '--', '-', '--', ':', '-.'])
    colors = cycle(['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])
    #cols = cycle(['room temperature', 'room temperature setpoint', 'flow', 'flow setpoint', 'damper position'])
    cols = cycle(['Room Temperature', 'Flow', 'Differential Pressure', 'Radiator Valve', 'Damper Position'])
    
    if (isSensorValue):
        for sensor in sensor_list:
            try:
                t, x = data[zone][sensor][0]
                x = x[:5040]
                t = t[:5040]
                ls= next(line_styles)
                color = next(colors)
                col = next(cols)
                
                ax.plot(t, x, linestyle = ls, label = col, color = color)

                numdays = math.floor(len(x)/(6*24))
                xticklabels = day_labels*int(numdays/7)+day_labels[0:(numdays%7)]

                plt.xticks(np.arange(0, len(x), 6*24), xticklabels, rotation=90)
                plt.xlim(-0.000001, len(x))
                ax.set_xlabel('Time', size = 'x-large')


                if(col == 'Temperature'):
                    ax.set_ylabel('Temperature (C)', size = 'x-large')
                elif(col == 'Flow'):
                    ax.set_ylabel('Flow (L/s)', size = 'x-large')
                elif(col == 'Differential Pressure'):
                    ax.set_ylabel('Differential Pressure (Pa)', size = 'x-large')
                elif(col == 'Radiator Valve'):
                    ax.set_ylabel('Radiator Valve', size = 'x-large')
                elif(col == 'Damper Position'):
                    ax.set_ylabel('Damper Position (%)', size = 'x-large')

            except:
                ls= next(line_styles)
                color = next(colors)
                col = next(cols)
                print('No data for' + sensor)
                
    else:
        try:
            x = data[int(zone)][0]
            t = tsdata[int(zone)]['TL1'][0]
            x = x[:1007]
            
            t = np.array(transpose(t))
            t = t[:1008]
            w1 = []
            for i in range(0, len(t)):
                w1.append(t[i])
            #print(t)
            w1 = w1[1:]
            ls= next(line_styles)
            color = next(colors)
            col = next(cols)

            ax.plot(w1, x, linestyle = ls, label = col, color = color)
            numdays = math.floor(len(x)/(6*24))
            xticklabels = day_labels*int(numdays/7)+day_labels[0:(numdays%7)]

            plt.xticks(np.arange(0, len(x), 6*24), xticklabels, rotation=90)
            plt.xlim(-0.000001, len(x))
            ax.set_xlabel('Time', size = 'x-large')
            ax.set_ylabel('Q', size = 'x-large')
        except:
            print('No data for room ' + zone)
            
    fig.tight_layout()
    fig.legend(bbox_to_anchor=(.92, .95), loc=1, borderaxespad=0.)
    plt.show()

In [ ]:
for room in room_list:
    if (not(room in black_list)):
        print("Room No. is: " + str(room))
        #print(tsdata[room])
        plotAllSensorData(room, tsdata, 1)

In [ ]:
X = {}
X_original = {}
for room in room_list:
    if (not(room in black_list)):
        X[room] = {}
        X_original[room]= {}
        for sensor in sensor_list:
            try:
                X[room][sensor] = []
                X_original[room][sensor] = []
                for data in range (0, len(tsdata[room][sensor][0][1])):
                    try:
                        X_original[room][sensor].append(tsdata[room][sensor][0][1][data])
                        if (data == 0):
                            X[room][sensor].append(tsdata[room][sensor][0][1][data] - np.mean(tsdata[room][sensor][0][1][data]))
                        else:
                            X[room][sensor].append(tsdata[room][sensor][0][1][data] - np.mean(tsdata[room][sensor][0][1][:data]))
                    except:
                        print('error is here')
            except:
                print('No data for ' + str(sensor) + ' in room ' + str(room))

In [ ]:
#######################################
### ASWANI'S METHOD FOR OBTAINING Q ###
#######################################

from numpy import *
from scipy.interpolate import *
from sklearn.linear_model import LinearRegression

coefs = {}
T_original = {}
V_original = {}
for room in room_list:
    if (not(room in black_list)):
        T = []
        V = []
        T_original[room] = []
        V_original[room] = []
        
        coefs[room] = []
        for sensor in sensor_list:
            if (sensor == 'TL1'):
                    T = X[room][sensor]
                    T_original[room] = X_original[room][sensor]
            else:
                if (len(X[room][sensor]) > 0):
                    if (len(V) > 0):
                        V = concatenate((V, [X[room][sensor][:-1]]))
                        V_original[room] = concatenate((V_original[room], [X_original[room][sensor][:-1]]))
                    else:
                        V = [X[room][sensor][:-1]]
                        V_original[room] = [X_original[room][sensor][:-1]]
        if (len(V) != 0):
            V = concatenate((V, [T[1:]]))
            V_original[room] = concatenate((V_original[room], [T_original[room][1:]]))
        else:
            V = [T[1:]]
            V_original[room] = [T_original[room][1:]]
      
        regressor = LinearRegression()
        Coffs = regressor.fit(transpose(V), T[:-1])
        coefs[room] = regressor.coef_

In [ ]:
q = {}
for room in room_list:
    if (not(room in black_list)):
        if (not(room in black_list)):
            #print(coefs[room])
            q[room] = []
            q[room].append(T_original[room][1:] - (transpose(coefs[room] * transpose(V_original[room])).sum(axis=0)))
        

In [ ]:
print("Naive Q Plotting")
for room in room_list:
    if (not(room in black_list)):
        print('Room No is: ' + str(room))
        #print(q[106][0])
        plotAllSensorData(str(room), q, 0)

In [ ]:
######################################
### EXCLUDING OUTSIDE WEATHER DATA ###
######################################
import requests

def GetWeatherData(day):
    url = "https://api.darksky.net/forecast/044b659c34748a0f15c31e5c2bc1cb9c/53.5444,-113.4909," + day + "?exclude=flags"
    response = requests.get(url)
    data = response.json()
    return data

In [ ]:
days = ['1485154800', '1485241200', '1485327600', '1485414000', '1485500400', '1485586800', '1485673200', \
        '1485759600', '1485846000', '1485932400', '1486018800', '1486105200', '1486191600', '1486278000', \
        '1486364400', '1486450800', '1486537200', '1486623600', '1486710000', '1486796400', '1486882800', \
        '1486969200', '1487055600', '1487142000', '1487228400', '1487314800', '1487401200', '1487487600', \
        '1487574000', '1487660400', '1487746800', '1487833200', '1487919600', '1488006000', '1488092400', \
        '1488178800', '1488265200', '1488351600', '1488438000', '1488524400', '1488610800', '1488697200']
#days = ['1485586800', '1485673200', '1485846000', '1485932400', '1486018800']

weather_data = []
for i in range(len(days)):
    weather_data.append(GetWeatherData(days[i]))

In [ ]:
tuned_data = []
for day in range(len(days)):
    for hour in range(0, 24):
        temp = weather_data[day]['hourly']['data'][hour]['temperature']
        wind = weather_data[day]['hourly']['data'][hour]['windSpeed']
        sunr = weather_data[day]['daily']['data'][0]['sunriseTime']
        suns = weather_data[day]['daily']['data'][0]['sunsetTime']
        
        try:
            cloud = weather_data[day]['hourly']['data'][hour]['cloudCover']
            
            try:
                for index in range(len(tuned_data)):
                    if(tuned_data[index][2] == -1):
                        tuned_data[index][2] = cloud;
            except:
                print('warning')
                
        except:
            cloud = -1
        
        for j in range(0, 6):
            tuned_data.append([temp, wind, cloud, sunr, suns])
            
import numpy as np
tuned_data = np.asarray(tuned_data)

In [ ]:
latitude = 53.5444
longitude = -113.4909

start_time = '2017-01-23 00:00:00'
end_time = '2017-03-06 00:00:00'

stime = start_time.split(' ')[0]
etime = end_time.split(' ')[0]

tus = pvlib.location.Location(latitude, longitude, 'America/Edmonton', 645, 'Edmonton')
times = pd.DatetimeIndex(start = start_time, end = end_time, freq='10min', tz=tus.tz)

ephem_data = tus.get_solarposition(times)
irrad_data = tus.get_clearsky(times)

sun_zen = ephem_data['apparent_zenith']
AM = pvlib.atmosphere.relativeairmass(sun_zen)
dni_et = pvlib.irradiance.extraradiation(times.dayofyear)

models = ['isotropic', 'klucher', 'haydavies', 'reindl', 'king', 'perez']
totals = {}

#for model in models:
model = 'isotropic'

total = pvlib.irradiance.total_irrad(60, 180, ephem_data['apparent_zenith'], ephem_data['azimuth'],
                                              dni=irrad_data['dni'], ghi=irrad_data['ghi'], dhi=irrad_data['dhi'],
                                              dni_extra=dni_et, airmass=AM,
                                              model=model,
                                              surface_type='urban')

solar_data = []

for i in range(len(total)):
     solar_data.append([total.iloc[i][0], total.iloc[i][1], 
                        total.iloc[i][2], total.iloc[i][3], 
                        total.iloc[i][4]])

solar_data = transpose(np.asarray(solar_data))    

In [ ]:
from numpy import *
from scipy.interpolate import *
from sklearn.linear_model import LinearRegression

coefs_p = {}
T_original_p = {}
V_original_p = {}
for room in room_list:
    if (not(room in black_list)):
        T_p = []
        V_p = []
        T_original_p[room] = []
        V_original_p[room] = []
        
        coefs_p[room] = []
        for sensor in sensor_list:
            if (sensor == 'TL1'):
                    T_p = X[room][sensor]
                    T_original_p[room] = X_original[room][sensor]
            else:
                if (len(X[room][sensor]) > 0):
                    if (len(V_p) > 0):
                        V_p = concatenate((V_p, [X[room][sensor][:-1]]))
                        V_original_p[room] = concatenate((V_original_p[room], [X_original[room][sensor][:-1]]))
                    else:
                        V_p = [X[room][sensor][:-1]]
                        V_original_p[room] = [X_original[room][sensor][:-1]]

        if (len(V_p) != 0):
            V_p = concatenate((V_p, [T_p[1:]]))
        else:
            V_p = [T_p[1:]]
            
        if (len(V_original_p[room]) != 0):
            V_original_p[room] = concatenate((V_original_p[room], [T_original_p[room][1:]]))
        else:
            V_original_p[room] = [T_original_p[room][1:]]

        V_p = vstack((V_p, tuned_data[:-1,0]))
        V_p = vstack((V_p, tuned_data[:-1,1]))
        V_p = vstack((V_p, tuned_data[:-1,2]))
           
        V_p = vstack((V_p, solar_data[0][:-2]))
        V_p = vstack((V_p, solar_data[1][:-2]))
        V_p = vstack((V_p, solar_data[2][:-2]))
        V_p = vstack((V_p, solar_data[3][:-2]))
        V_p = vstack((V_p, solar_data[4][:-2]))
        
        V_original_p[room] = vstack((V_original_p[room], tuned_data[:-1,0]))
        V_original_p[room] = vstack((V_original_p[room], tuned_data[:-1,1]))
        V_original_p[room] = vstack((V_original_p[room], tuned_data[:-1,2]))
        
        V_original_p[room] = vstack((V_original_p[room], solar_data[0][:-2]))
        V_original_p[room] = vstack((V_original_p[room], solar_data[1][:-2]))
        V_original_p[room] = vstack((V_original_p[room], solar_data[2][:-2]))
        V_original_p[room] = vstack((V_original_p[room], solar_data[3][:-2]))
        V_original_p[room] = vstack((V_original_p[room], solar_data[4][:-2]))
        
        regressor = LinearRegression()
        Coffs = regressor.fit(transpose(V_p), T_p[:-1])
        coefs_p[room] = regressor.coef_

In [ ]:
q_p = {}
for room in room_list:
    if (not(room in black_list)):
        #if (not(room in interior_offices_private or room in corridors)):
        #if (not(room in interior_offices_private)):
        if (True):
            q_p[room] = []
            q_p[room].append(T_original_p[room][1:] - (transpose(coefs_p[room] * transpose(V_original_p[room])).sum(axis=0)))
        else:
            q_p[room] = []
            q_p[room].append(q[room][0])

In [ ]:
print("Ploting Q, excluding weather effects")
for room in room_list:
    if (not(room in black_list)):
        print(room)
        # print('room no. is: ' + str(room) + ' Interior zone? ' + str((room in interior_offices_private or room in corridors)))
        plotAllSensorData(str(room), q_p, 0)

In [ ]:
# plotting
import numpy as np

matplotlib.rcParams['figure.figsize'] = (30.0, 10.0)

colors = {'TL1':'r', 'TL3':'--m', 'TL5':'b', 'TL8':'--g', 'TL6':':c'}

def ClusterPlot(zone, data):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    
    # Move the last y-axis spine over to the right by x% of the width of the axes
    ax3.spines['right'].set_position(('axes', 1.05))
    ax3.set_frame_on(True)
    ax3.patch.set_visible(False)
    ax3.yaxis.set_major_formatter(matplotlib.ticker.OldScalarFormatter())
    
    lines = []

    x = data[zone]
    t = tsdata[zone]['TL1'][0]
    t = t[0][:1008]
    #t = np.array(transpose(t))
    #print(t)
    w1 = []
    for i in range(0, len(t)):
        w1.append(t[i])
    
    #w1 = w1[1:]
    
    w2 = []
    for i in range(0, len(x[0])):
        w2.append(x[0][i])

    w2 = w2[:1008]
    
    ax.plot(w1, w2, linestyle = '-.', label = 'Cluster Label', color = 'C9')

    numdays = math.floor(len(w2)/(6*24))
    xticklabels = day_labels*int(numdays/7)+day_labels[0:(numdays%7)]

    plt.xticks(np.arange(0, len(w2), 6*24), xticklabels, rotation=90)
    plt.xlim(-0.000001, len(w2))
    plt.ylim(min(w2), max(w2))
    ax.set_xlabel('Time', size = 'x-large')
    ax.set_ylabel('Cluster Label', size = 'x-large')

    
    ####################
    fig, ax = plt.subplots()
    ax2 = ax.twinx()
    ax3 = ax.twinx()

    # Move the last y-axis spine over to the right by x% of the width of the axes
    ax3.spines['right'].set_position(('axes', 1.05))
    ax3.set_frame_on(True)
    ax3.patch.set_visible(False)
    ax3.yaxis.set_major_formatter(matplotlib.ticker.OldScalarFormatter())

    lines = []
    line_styles = cycle(['-', '--', '-', '--', ':', '-.'])
    colors = cycle(['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])
    #cols = cycle(['room temperature', 'room temperature setpoint', 'flow', 'flow setpoint', 'damper position'])
    cols = cycle(['Room Temperature', 'Flow', 'Differential Pressure', 'Radiator Valve', 'Damper Position'])


    for sensor in sensor_list:
        try:
            t, x = tsdata[zone][sensor][0]
            
            t = t[:1008]
            x = x[:1008]
            
            ls= next(line_styles)
            color = next(colors)
            col = next(cols)

            ax.plot(t, x, linestyle = ls, label = col, color = color)

            numdays = math.floor(len(x)/(6*24))
            xticklabels = day_labels*int(numdays/days_number)+day_labels[0:(numdays%days_number)]

            plt.xticks(np.arange(0, len(x), 6*24), xticklabels, rotation=90)
            plt.xlim(-0.000001, len(x))
            ax.set_xlabel('Time', size = 'x-large')


            if(col == 'Temperature'):
                ax.set_ylabel('Temperature (C)', size = 'x-large')
            elif(col == 'Flow'):
                ax.set_ylabel('Flow (L/s)', size = 'x-large')
            elif(col == 'Differential Pressure'):
                ax.set_ylabel('Differential Pressure (Pa)', size = 'x-large')
            elif(col == 'Radiator Valve'):
                ax.set_ylabel('Radiator Valve', size = 'x-large')
            elif(col == 'Damper Position'):
                ax.set_ylabel('Damper Position (%)', size = 'x-large')

        except:
            ls= next(line_styles)
            color = next(colors)
            col = next(cols)
            print('No data for' + sensor)

    fig.tight_layout()
    fig.legend(bbox_to_anchor=(.92, .95), loc=1, borderaxespad=0.)
    plt.show()

In [ ]:
# plotting
import numpy as np

matplotlib.rcParams['figure.figsize'] = (30.0, 10.0)

colors = {'TL1':'r', 'TL2':'--m', 'TL3':'b', 'TL4':'--g', 'TL6':':c'}

def FuzzyClusterPlot(zone):
    
    fig, ax = plt.subplots()
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    
    # Move the last y-axis spine over to the right by x% of the width of the axes
    ax3.spines['right'].set_position(('axes', 1.05))
    ax3.set_frame_on(True)
    ax3.patch.set_visible(False)
    ax3.yaxis.set_major_formatter(matplotlib.ticker.OldScalarFormatter())
    
    lines = []
    line_styles = cycle(['-', '--', '-', '--', ':', '-.'])
    colors = cycle(['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])
    cols = cycle(['cluster label', 'room temperature', 'room temperature setpoint', 'flow', 'flow setpoint', 'damper position'])

    x = result[zone]
   # x = np.array(q[zone][0])
    t = tsdata[zone]['TL1'][0]
    #t = np.array(transpose(t[:-1]))
    t = np.array(transpose(t))
    
    x = x[:1008]
    t = t[:1008]
    
    w1 = []
    for i in range(0, len(t)):
        w1.append(t[i])
    
    #w1 = w1[1:]
    
    w2 = []
    for i in range(0, len(x[0])):
        w2.append(x[0][i])
    
    ls= next(line_styles)
    color = next(colors)
    col = next(cols)

    print(col);
    
    ax.plot(w1, w2, linestyle = ls, label = col, color = color)

    numdays = math.floor(len(w2)/(6*24))
    xticklabels = day_labels*int(numdays/7)+day_labels[0:(numdays%7)]

    plt.xticks(np.arange(0, len(w2), 6*24), xticklabels, rotation=90)
    plt.xlim(-0.000001, len(w2))
    plt.ylim(min(w2), max(w2))
    ax.set_xlabel('Time', size = 'x-large')
    ax.set_ylabel('Temperature (C)', size = 'x-large')
    
            
    fig.tight_layout()
    fig.legend(bbox_to_anchor=(.92, .95), loc=1, borderaxespad=0.)
    plt.show()

In [ ]:
################################################
### Using Clustering to estimate occupancies ###
################################################

#Input: Sensors (['TL1', 'TL3', 'TL5', 'TL8', 'TL6'])
#Output: 1 = one or more peaple | 0 = no occupancies

#Training Data: Weekend Data
#Labeling: Using 1) Clustering

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

weekend_weight = 1
weekday_weight = 1
number_of_days = days_number
number_of_classes = 2

clustering_data = {}

weight = np.ones(number_of_days * 144)

weight[:244] = weight[:244] * weekend_weight
weight[244:len(weight)] = weight[244:len(weight)] * weekday_weight


for room in room_list:
    if (not(room in black_list)):
        counter = 0
        clustering_data[room] = []
        for sensor in sensor_list:
            try:
                #print(tsdata[room][sensor][1])
                clustering_data[room].append(tsdata[room][sensor][0][1])

            except:
                print("no data for sensor " + str(sensor) + " in room no. " + str(room))

cluster = KMeans(n_clusters = number_of_classes)


#print(tsdata[106])
result = {}
for room in room_list:
    if (not(room in black_list)):
        result[room] = []
        result[room].append(cluster.fit_predict(transpose(clustering_data[room])))

for room in room_list:
    if (not(room in black_list)):
        #print(str(room))
        ClusterPlot(room, result)


In [ ]:
#######################################################
### Using  Fuzzy Clustering to estimate occupancies ###
#######################################################

#Input: Sensors (['TL1', 'TL3', 'TL5', 'TL8', 'TL6'])
#Output: 1 = one or more peaple | 0 = no occupancies

#Training Data: Weekend Data
#Labeling: Using 1) Clustering

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import skfuzzy as fuzz

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

weekend_weight = 1
weekday_weight = 1
number_of_days = 7
number_of_classes = 2
exponentiation = 2
error = 0.005
maxiter = 1000
init = None

clustering_data = {}

weight = np.ones(number_of_days * 144)

weight[:244] = weight[:244] * weekend_weight
weight[244:len(weight)] = weight[244:len(weight)] * weekday_weight


for room in room_list:
    if (not(room in black_list)):
        counter = 0
        clustering_data[room] = []
        for sensor in sensor_list:
            try:
                clustering_data[room].append(tsdata[room][sensor][0][1])

            except:
                print("no data for sensor " + str(sensor) + " in room no. " + str(room))

fuzzy_result = {}
for room in room_list:
    if (not(room in black_list)):
        fuzzy_result[room] = []
        cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(np.array(clustering_data[room]), number_of_classes, exponentiation, error, maxiter, init)
        sum1 = sum(u[1])
        sum2 = sum(u[0])

        if (sum1 <= sum2):
            index = 1
        else:
            index = 0

        fuzzy_result[room].append(u[index])

for room in room_list:
    if (not(room in black_list)):
        # print('room no. is: ' + str(room) + ' Interior zone? ' + str((room in interior_offices_private or room in corridors)))
        ClusterPlot(room, fuzzy_result)

In [ ]:
###########################################
### Confidence Level based on room type ###
###########################################
import math

def ConfidenceLevel(values):
    dist = 0;
    temp = [x-0.5 for x in values]
    raw = [abs(number) for number in temp]
    norm = [(float(i) - min(raw))/(max(raw) - min(raw) + 1e-30) for i in raw]
    if not(math.isnan(norm[0])):
        return (sum(norm)/len(norm))
    else:
        # print(raw)
        return 0

In [ ]:
import numpy as np

confis = []
for room in room_list:
    if (not(room in black_list)):
        confis.append(ConfidenceLevel(fuzzy_result[room][0]))
print('All rooms average Confidence is: ' + str(sum(confis)/len(confis)))
print('All rooms Confidence STD is: ' + str(np.std(confis)) + '\n-----------\n')

# confis = []
# for room in corridors:
#     if (not(room in black_list)):
#         confis.append(ConfidenceLevel(fuzzy_result[room][0]))
# print('Corridors average Confidence is: ' + str(sum(confis)/len(confis)))
# print('Corridors Confidence STD is: ' + str(np.std(confis)) + '\n-----------\n')

# confis = []
# for room in interior_offices_private:
#     if (not(room in black_list)):
#         confis.append(ConfidenceLevel(fuzzy_result[room][0]))
# print('Interior offices average Confidence is: ' + str(sum(confis)/len(confis)))
# print('Interior offices Confidence STD is: ' + str(np.std(confis)) + '\n-----------\n')

# confis = []
# for room in exterior_offices_private:
#     if (not(room in black_list)):
#         confis.append(ConfidenceLevel(fuzzy_result[room][0]))
# print('Exterior offices average Confidence is: ' + str(sum(confis)/len(confis)))
# print('Exterior offices Confidence STD is: ' + str(np.std(confis)) + '\n-----------\n')

# confis = []
# for room in facility_rooms:
#     if (not(room in black_list)):
#         confis.append(ConfidenceLevel(fuzzy_result[room][0]))
# print('Facility rooms average Confidence is: ' + str(sum(confis)/len(confis)))
# print('Facility rooms Confidence STD is: ' + str(np.std(confis)) + '\n-----------\n')

# confis = []
# for room in shared_places:
#     if (not(room in black_list)):
#         confis.append(ConfidenceLevel(fuzzy_result[room][0]))
# print('Shared places average Confidence is: ' + str(sum(confis)/len(confis)))
# print('Shared places Confidence STD is: ' + str(np.std(confis)) + '\n-----------\n')

In [ ]:
#######################################################################
### Using few classification algorithms based on clustering results ###
#######################################################################

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
import keras.utils
import numpy as np
import pprint
import math
import keras
from keras.optimizers import SGD


training_chunk = 4032
validation_chunk = 576

result2 = {}
for room in room_list:
    if (not(room in black_list)):
        result2[room] = []
        for i in range(len(fuzzy_result[room][0])):
            result2[room].append(int(fuzzy_result[room][0][i] * 10))

history = {}
evaluations = {}

for room in room_list:
    if (not(room in black_list)):
        print(room)
        data_count = len(result2[room])
        features_count = len(clustering_data[room])
        classes_count = math.ceil(max(result2[room])*10)

        history[room] = []
        labels_train = keras.utils.to_categorical(result2[room][:training_chunk], num_classes=classes_count)
        labels_test = keras.utils.to_categorical(result2[room][training_chunk:], num_classes=classes_count)

        X_train = transpose(np.asarray(clustering_data[room]))[:training_chunk]
        X_test = transpose(np.asarray(clustering_data[room]))[training_chunk:]
        evaluations[room] = []

        model = Sequential()
        model.add(Dense(features_count, activation='relu', input_dim=features_count))
        model.add(Dense(classes_count, activation='softmax'))
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        history[room] = model.fit(X_train, labels_train, epochs=20, batch_size=128)
        evaluations[room] = model.evaluate(X_test, labels_test, batch_size=128)
        

In [ ]:
for room in room_list:
    if (not(room in black_list)):
        print(room)
        print('\t[loss, accuracy] on test data: ' + str(evaluations[room]))

In [ ]:
import numpy as np

accuracies = {}
for room in room_list:
    if (not(room in black_list)):
        accuracies[room] = []
        accuracies[room].append(evaluations[room][1])

acc = []
for room in corridors:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Corridors mean is: ' + str(sum(acc)/len(acc)))
print('Corridors STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in interior_offices_private:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Interior offices mean is: ' + str(sum(acc)/len(acc)))
print('Interior offices STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in exterior_offices_private:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Exterior offices mean is: ' + str(sum(acc)/len(acc)))
print('Exterior offices STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in facility_rooms:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Facility rooms mean is: ' + str(sum(acc)/len(acc)))
print('Facility rooms STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in shared_places:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Shared places mean is: ' + str(sum(acc)/len(acc)))
print('Shared places STD is: ' + str(np.std(acc)) + '\n-----------\n')

In [ ]:
########################################################
### Designing a classifier to predict the Aswani's Q ###
########################################################

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
import keras.utils
import numpy as np
import pprint
import math
import keras
from keras.optimizers import SGD


########################################################

training_chunk = 4032
validation_chunk = 576

result2 = {}
for room in room_list:
    if (not(room in black_list)):
        result2[room] = q_p[room][0]

history = {}
evaluations = {}

for room in room_list:
    if (not(room in black_list)):
        print(room)
        data_count = len(result2[room])
        features_count = len(clustering_data[room])
        classes_count = math.ceil(max(result2[room])*10)

        history[room] = []
        labels_train = keras.utils.to_categorical(result2[room][:training_chunk], num_classes=classes_count)
        labels_test = keras.utils.to_categorical(result2[room][training_chunk:], num_classes=classes_count)

        X_train = transpose(np.asarray(clustering_data[room]))[:training_chunk]
        X_test = transpose(np.asarray(clustering_data[room]))[training_chunk:-1]
        evaluations[room] = []

        model = Sequential()
        model.add(Dense(features_count, activation='relu', input_dim=features_count))
        model.add(Dense(classes_count, activation='softmax'))
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        history[room] = model.fit(X_train, labels_train, epochs=20, batch_size=128)
        evaluations[room] = model.evaluate(X_test, labels_test, batch_size=128)
        

In [ ]:
import numpy as np

accuracies = {}
for room in room_list:
    if (not(room in black_list)):
        accuracies[room] = []
        accuracies[room].append(evaluations[room][1])

acc = []
for room in corridors:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Corridors mean is: ' + str(sum(acc)/len(acc)))
print('Corridors STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in interior_offices_private:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Interior offices mean is: ' + str(sum(acc)/len(acc)))
print('Interior offices STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in exterior_offices_private:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Exterior offices mean is: ' + str(sum(acc)/len(acc)))
print('Exterior offices STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in facility_rooms:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Facility rooms mean is: ' + str(sum(acc)/len(acc)))
print('Facility rooms STD is: ' + str(np.std(acc)) + '\n-----------\n')

acc = []
for room in shared_places:
    if (not(room in black_list)):
        acc.append(accuracies[room])
print('Shared places mean is: ' + str(sum(acc)/len(acc)))
print('Shared places STD is: ' + str(np.std(acc)) + '\n-----------\n')

In [ ]:
for room in room_list:
    if (not(room in black_list)):
        print(room)
        print('\t[loss, accuracy] on test data: ' + str(evaluations[room]))

In [ ]:
##################################################################
### Average of fuzzy clustering to infer occupancies in a week ###
##################################################################
import numpy as np
number_of_weeks = 6

occ = {}
for room in room_list:
    if (not(room in black_list)):
        occ[room] = [np.zeros(1008)]
        for i in range(len(fuzzy_result[room][0])):
            occ[room][0][i%1008] += fuzzy_result[room][0][i]

        occ[room][0][:] = [x / number_of_weeks for x in occ[room][0]]

In [ ]:
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
print(occ[306])

In [ ]:
for room in room_list:
    if (not(room in black_list)):
        plotAllSensorData(room, occ, 0)